In [ ]:
import pandas as pd
import os
from google.colab import drive
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import numpy as np
from io import StringIO
from scipy.stats import mannwhitneyu
from scipy.stats import pearsonr
from scipy.stats import spearmanr
from matplotlib.ticker import MultipleLocator

# OPTIONAL: Load data from local or online source
# If running on Colab and using Google Drive, uncomment below:

# from google.colab import drive
# drive.mount('/content/drive')
# os.chdir('/content/drive/MyDrive/your_project_folder')

# Otherwise, place your data in the same folder or set the correct relative path

In [ ]:
pca_recent   = pd.read_csv("data/pca_normalized_recent.csv")
kpca_recent  = pd.read_csv("data/kpca_normalized_recent.csv")
wpca_recent  = pd.read_csv("data/wpca_normalized_recent.csv")

In [ ]:
url = "https://www.cpc.ncep.noaa.gov/data/indices/oni.ascii.txt"
response = requests.get(url)
raw_text = response.text
df = pd.read_csv(StringIO(raw_text), sep='\s+')

enso_df = (
    df
    .groupby('YR', as_index=False)
    .agg({'ANOM': 'mean'})
    .rename(columns={'YR': 'Year_Bin', 'ANOM': 'ENSO'})
)

enso_df['ENSO_norm'] = (enso_df['ENSO'] - enso_df['ENSO'].mean()) / enso_df['ENSO'].std()
enso_df = enso_df[enso_df['Year_Bin'] <= 2016]
enso_df.head()

In [ ]:
pdo_df = pd.read_csv('data/pdo.timeseries.ersstv5.csv')
pdo_df['Date'] = pd.to_datetime(pdo_df['Date'])
pdo_df['Year_Bin'] = pdo_df['Date'].dt.year
pdo_df = pdo_df.rename(columns={pdo_df.columns[1]: 'PDO'})
pdo_df = pdo_df.groupby('Year_Bin')['PDO'].mean().reset_index()
pdo_df['PDO_norm'] = (pdo_df['PDO'] - pdo_df['PDO'].mean()) / pdo_df['PDO'].std()
pdo_df = pdo_df[pdo_df['Year_Bin'] <= 2016]
pdo_df.head()

In [ ]:
# Step 1: Load and clean the data
data = []
with open("data/composite_d25_07_0310a.dat", "r") as f:
    for i, line in enumerate(f):
        if i < 42:
            continue  # Skip header
        parts = line.strip().split()
        if len(parts) != 3:
            continue
        date, day_of_epoch, irradiance = parts
        irradiance = float(irradiance)
        if irradiance == -99.0000:
            continue  # Skip missing values
        data.append((date, float(day_of_epoch), irradiance))  # <-- fix here

# Step 2: Convert to DataFrame
df = pd.DataFrame(data, columns=["Date", "DayOfEpoch", "Irradiance"])

# Step 3: Extract full year from YYMMDD
def parse_year(ymd):
    y = int(ymd[:2])
    return 1900 + y if y >= 78 else 2000 + y

df["YR"] = df["Date"].apply(parse_year)

# Step 4: Group and bin
tsi_df = (
    df.groupby("YR", as_index=False)
    .agg({"Irradiance": "mean"})
    .rename(columns={'YR': 'Year_Bin', 'Irradiance': 'TSI'})
)

tsi_df['TSI_norm'] = (tsi_df['TSI'] - tsi_df['TSI'].mean()) / tsi_df['TSI'].std()
tsi_df = tsi_df[tsi_df['Year_Bin'] <= 2016]
tsi_df.head()

In [ ]:
co2_df = pd.read_csv('data/co2_gr_mlo.csv', skiprows=45)
co2_df = co2_df.rename(columns={'year': 'Year_Bin', 'ann inc': 'CO2'})
co2_df['CO2_norm'] = (co2_df['CO2'] - co2_df['CO2'].mean()) / co2_df['CO2'].std()
co2_df = co2_df[co2_df['Year_Bin'] <= 2016]
co2_df.head()

In [ ]:
# Compute median and 5–95 percentile for each PCA
def summarize(df, colname):
    grouped = df.groupby("Year_Bin")[colname]
    median = grouped.median()
    p5 = grouped.quantile(0.05)
    p95 = grouped.quantile(0.95)
    return pd.DataFrame({
        "Year_Bin": median.index,
        "median": median.values,
        "p5": p5.values,
        "p95": p95.values
    })

pca_summary = summarize(pca_recent, "PC1_norm")
pca_summary = pca_summary[pca_summary["Year_Bin"] >= 1950]

kpca_summary = summarize(kpca_recent, "KPC1_norm")
kpca_summary = kpca_summary[kpca_summary["Year_Bin"] >= 1950]

wpca_summary = summarize(wpca_recent, "WPC1_norm")
wpca_summary = wpca_summary[wpca_summary["Year_Bin"] >= 1950]

In [ ]:
fig, axs = plt.subplots(5, 1, figsize=(14, 13), sharex=True, gridspec_kw={'height_ratios': [2, 1, 1, 1, 1]})

for ax in axs:
    ax.xaxis.set_major_locator(MultipleLocator(20))
    ax.tick_params(labelbottom=True)
    ax.grid(True, axis='x', linestyle='--', color='gray', linewidth=0.8, alpha=0.6)

# Panel 1: Hydroclimate Reconstruction
for summary, label, color in zip([pca_summary, kpca_summary, wpca_summary],
                                 ['Standard PCA', 'Kernel PCA', 'Wavelet PCA'],
                                 ['blue', 'green', 'orange']):
    axs[0].plot(summary["Year_Bin"], summary["median"], label=label, color=color)
    axs[0].fill_between(summary["Year_Bin"], summary["p5"], summary["p95"], alpha=0.2, color=color)

axs[0].set_ylabel("PC1 (Normalized)", fontsize=12)
axs[0].set_title("Hydroclimate Variability Reconstruction", fontsize=14)
axs[0].legend()

# Volcanic eruption years
eruption_years = [1963, 1991]

# Set y-position for triangle markers (just below the plot)
eruption_y = axs[0].get_ylim()[0] - 0.05  # adjust this if needed

# Plot upright red triangles
axs[0].scatter(
    eruption_years,
    [eruption_y] * len(eruption_years),
    marker='^', color='red', s=80, label='Volcanic Eruption'
)

# Adjust y-axis limit to make space for the triangles
axs[0].set_ylim(eruption_y - 0.1, axs[0].get_ylim()[1])

# Make sure it's added to the existing legend
axs[0].legend(loc='upper left', fontsize=12)


# Panel 2: ENSO
axs[1].plot(enso_df["Year_Bin"], enso_df["ENSO_norm"], color='darkcyan')
axs[1].set_ylabel("ENSO (Normalized)", fontsize=12)
axs[1].set_title("Observed ENSO", fontsize=14)


# Panel 3: PDO
axs[2].plot(pdo_df["Year_Bin"], pdo_df["PDO_norm"], color='brown')
axs[2].set_ylabel("PDO (Normalized)", fontsize=12)
axs[2].set_title("Observed PDO", fontsize=14)


# Panel 4: TSI
axs[3].plot(tsi_df["Year_Bin"], tsi_df["TSI_norm"], color='crimson')
axs[3].set_ylabel("TSI (Normalized)", fontsize=12)
axs[3].set_title("Observed TSI", fontsize=14)

# Panel 5: CO2
axs[4].plot(co2_df["Year_Bin"], co2_df["CO2_norm"], color='magenta')
axs[4].set_ylabel("CO2 (Normalized)", fontsize=12)
axs[4].set_xlabel("Year (CE)", fontsize=12)
axs[4].set_title("Observed CO2", fontsize=14)

for ax in axs:
    ax.set_xlim(1950, 2016)

plt.tight_layout()
plt.show()

In [ ]:
pca_methods = {
    'Standard PCA': (pca_recent, 'PC1_norm'),
    'Kernel PCA': (kpca_recent, 'KPC1_norm'),
    'Wavelet PCA': (wpca_recent, 'WPC1_norm')
}

forcing_datasets = {
    'ENSO': enso_df[['Year_Bin', 'ENSO_norm']].rename(columns={'ENSO_norm': 'forcing'}),
    'PDO': pdo_df[['Year_Bin', 'PDO_norm']].rename(columns={'PDO_norm': 'forcing'}),
    'TSI': tsi_df[['Year_Bin', 'TSI_norm']].rename(columns={'TSI_norm': 'forcing'}),
    'CO2': co2_df[['Year_Bin', 'CO2_norm']].rename(columns={'CO2_norm': 'forcing'})
}

# Analysis function
def analyze_correlation(forcing_name, forcing_df):
    fig, axs = plt.subplots(1, 3, figsize=(14, 4))

    for ax, (label, (pca_df, pc_col)) in zip(axs, pca_methods.items()):
        pca_df_filtered = pca_df[pca_df['Year_Bin'] >= 1950]
        forcing_df_filtered = forcing_df[forcing_df['Year_Bin'] >= 1950]
        merged = pd.merge(pca_df_filtered, forcing_df_filtered, on='Year_Bin', how='inner')

        observed_corrs = []
        null_corrs = []

        for r in merged['Realization'].unique():
            subset = merged[merged['Realization'] == r]
            corr_actual = np.corrcoef(subset[pc_col], subset['forcing'])[0,1]
            observed_corrs.append(corr_actual)

            for _ in range(50):
                shuffled_pc1 = np.random.permutation(subset[pc_col])
                corr_null = np.corrcoef(shuffled_pc1, subset['forcing'])[0,1]
                null_corrs.append(corr_null)

        # Plot
        sns.kdeplot(null_corrs, ax=ax, color='grey', linestyle='--', label='Null Distribution (Shuffled)')
        sns.kdeplot(observed_corrs, ax=ax, color='blue', label='Actual Correlations')

        # Add medians
        ax.axvline(np.median(null_corrs), color='grey', linestyle='dotted')
        ax.axvline(np.median(observed_corrs), color='blue', linestyle='solid', linewidth=1)

        # Mann-Whitney U Test
        stat, p_val = mannwhitneyu(observed_corrs, null_corrs, alternative='two-sided')

        # Legend and p-value positioning
        ax.legend(loc='upper left', fontsize=10)
        ax.text(0.05, 0.75, f'Mann–Whitney p = {p_val:.4f}', transform=ax.transAxes, fontsize=10,
                bbox=dict(facecolor='white', edgecolor='black', boxstyle='round,pad=0.3'))

        ax.set_title(f'{label} - {forcing_name}')
        ax.set_xlabel('Pearson Correlation', fontsize=12)
        ax.set_ylabel('Density', fontsize=12)
        ax.tick_params(axis='both', labelsize=10)

    fig.suptitle(f'Comparison of Observed vs Null Correlations: {forcing_name}', fontsize=14)
    fig.tight_layout(rect=[0, 0, 1, 0.98])
    plt.show()

# Run analysis for each forcing
for forcing_name, forcing_df in forcing_datasets.items():
    analyze_correlation(forcing_name, forcing_df)